# Исследование надежности заемщиков.

Целью данного исследования является <span style = "color:blue">*выяснить, влияет ли семейное положение заемщика и наличие у него детей на выплату займа в срок* </span>. Эта информация будет учтена при построении модели скоринга, по которой можно понять, находится ли клиент в какой-либо группе риска и как этот риск велик.

**План работ:**
1. Просмотреть данные и выявить ошибки: аномалии, дубликаты, пропуски.
2. Исправить ошибки для более корректного анализа данных.
3. Проанализировать данные и понять, есть ли зависимость выплаты займа в срок от факторов наличия детей и семейного положения заемщика.
4. Предоставить выводы по результатам исследования и, по возможности, дать дополнительные рекомендации.

### Шаг 1. Обзор данных

In [5]:
import pandas as pd
stat = pd.read_csv('data.csv') # сохраняю данные в датасет stat
stat.info() # первый взгляд на датасет

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.1 Заполнение пропусков

In [6]:
# смотрим что за пропуски в столбцах и сколько их

skipped_values_days = stat['days_employed'].isna().sum() # чуть усложним код для красивого и понятного вывода
skipped_values_income = stat['total_income'].isna().sum()
skipped_part = skipped_values_income / len(stat['total_income'])
print('Пропущено значений days_employed', skipped_values_days)
print('Пропущено значений total_income', skipped_values_income)
print(f'Доля пропущенных значений равна {skipped_part:.1%}')

print(stat['days_employed'][stat['total_income'].isna()].count()) # проверяю есть ли такие строки, где при отсутствующем значении дохода есть значение стажа

Пропущено значений days_employed 2174
Пропущено значений total_income 2174
Доля пропущенных значений равна 10.1%
0


В таблице есть пропуски в столбцах  `days_employed` и `total_income`. 
Количество пропусков одинаково и равно 2174. Доля же пропусков составляет чуть более **10%**, что достаточно много.
Более того, есть закономерность в пропусках: где не указан доход, там и не указан стаж.

**Вероятные причины наличия пропусков:**
- люди не указывали доход и стаж
- техническая ошибка
- люди без стажа и дохода (частный случай п.1)

Для `total_income` можно заполнить пропуски медианными значениями. Среднее не подойдет, т.к. могут быть всплески (слишком высокие или слишком низкие значения), которые сильно повлияют на общую картину.

In [7]:
# заполняю пропуски в total_income

poor = stat.groupby('income_type') # группирую датасет по типу занятости
stat['total_income'] = poor.total_income.apply(lambda x: x.fillna(x.median())) # применяю медианное значение по группам к пропускам
stat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.2 Проверка данных на аномалии и исправления.

In [8]:
# сперва проверим столбцы на аномалии

for column in stat.columns:
    print(stat[column].value_counts())
    print('*'*60)

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
************************************************************
-8437.673028      1
-3507.818775      1
 354500.415854    1
-769.717438       1
-3963.590317      1
                 ..
-1099.957609      1
-209.984794       1
 398099.392433    1
-1271.038880      1
-1984.507589      1
Name: days_employed, Length: 19351, dtype: int64
************************************************************
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
0     

**Видим аномалии в:**
1. Наличие детей (есть отрицательные значения и есть семьи с 20 детьми).
2. Возраст заемщика (101 заемщик, которым нет еще и года).
3. Трудовой стаж (отрицательные значения и стаж в 1000+ лет).
4. Образование (тут просто дубликаты).
5. Пол (один заемщик не определился с полом).
6. Цель (тут тоже просто дубликаты).

**Возможные причины:**
1. Человеческий фактор. Например, случайно добавили 0 к двум детям или не указали пол.
2. Наличие дубликатов - причина делать выпадающие списки в интерфейсах, а не просто строку ввода.
3. В трудовом стаже скорее всего техническая проблема, потому как указать отрицательный стаж руками в таких количествах или же ошибиться на 2-3 знака очень сложно.

In [9]:
# разберемся с детьми

stat.loc[stat['children'] == -1, 'children'] = 1 # меняю отрицательные значения
stat.loc[stat['children'] == 20, 'children'] = 2 # привожу значения ближе к дейстивтельности
stat['children'].value_counts() # теперь похоже на правду

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [10]:
# теперь заменю возраст заемщиков с ошибкой на среднее значение возраста. В данном случее среднее подойдет

stat.loc[stat['dob_years'] == 0, 'dob_years'] = stat['dob_years'].mean()
stat['dob_years'].value_counts() # проверяю, что все ок

35.00000    617
40.00000    609
41.00000    607
34.00000    603
38.00000    598
42.00000    597
33.00000    581
39.00000    573
31.00000    560
36.00000    555
44.00000    547
29.00000    545
30.00000    540
48.00000    538
37.00000    537
50.00000    514
43.00000    513
32.00000    510
49.00000    508
28.00000    503
45.00000    497
27.00000    493
56.00000    487
52.00000    484
47.00000    480
54.00000    479
46.00000    475
58.00000    461
57.00000    460
53.00000    459
51.00000    448
59.00000    444
55.00000    443
26.00000    408
60.00000    377
25.00000    357
61.00000    355
62.00000    352
63.00000    269
64.00000    265
24.00000    264
23.00000    254
65.00000    194
22.00000    183
66.00000    183
67.00000    167
21.00000    111
43.29338    101
68.00000     99
69.00000     85
70.00000     65
71.00000     58
20.00000     51
72.00000     33
19.00000     14
73.00000      8
74.00000      6
75.00000      1
Name: dob_years, dtype: int64

In [11]:
# время разобраться со стажем

stat['days_employed'] = stat['days_employed'].apply(abs) # меняю знак отрицательных значений
print(stat[stat['days_employed'] < 0]['days_employed'].count()) # проверка

print(stat[stat['days_employed'] > 14600]['days_employed'].count()) # смотрю сколько людей со стажем более 40 лет
stat.loc[stat['days_employed'] >= 14600, 'days_employed'] = stat['days_employed'].median() # заменяю такие значения на медианные
stat['days_employed'] = stat['days_employed'].fillna(stat['days_employed'].median()) # заполняю пустые значения
print(stat[stat['days_employed'] > 14600]['days_employed'].count()) # one more check

0
3462
0


In [12]:
# и наконец дропну/отфильтрую строку с гендерно-нейтральным заемщиком
stat = stat.drop(stat.loc[stat['gender'] == 'XNA'].index)
print(stat['gender'].value_counts())

F    14236
M     7288
Name: gender, dtype: int64


Я исправил ряд значений в столбцах, для более точного анализа датасета. Осталась одна аномалия, которая может таковой и не являться: в таблице есть пара тысяч записей о заемщиках, чей средний возраст составляет почти 40 лет, но при этом стаж менее 3 лет (в ряде случаев менее года). Их тип занятости при этом в большинстве случаев именно наемный рабочий.
Тут я делаю допущение, что такая картина может быть вполне реальной, и оставляю записи без изменений, как минимум потому что точность этих данных не критична для решения поставленной задачи.

### Шаг 2.3. Изменение типов данных.

Изменим тип данных в таблице:
- в столбце `total_income`
- в столбце `dob_years` (изменил ранее при фиксе аномалий)

In [13]:
stat['total_income'] = stat['total_income'].astype('int')
stat['dob_years'] = stat['dob_years'].astype('int')
stat.info() # проверяем типы данных

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21524 non-null  int64  
 1   days_employed     21524 non-null  float64
 2   dob_years         21524 non-null  int64  
 3   education         21524 non-null  object 
 4   education_id      21524 non-null  int64  
 5   family_status     21524 non-null  object 
 6   family_status_id  21524 non-null  int64  
 7   gender            21524 non-null  object 
 8   income_type       21524 non-null  object 
 9   debt              21524 non-null  int64  
 10  total_income      21524 non-null  int64  
 11  purpose           21524 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


### Шаг 2.4. Удаление дубликатов.

Необходимо отработать дубликаты, найденные ранее в столбце ```education```.
Дубликаты в ```purpose``` удалять не будем, а позднее категоризируем.

In [14]:
print(stat.duplicated().sum()) # смотрю сколько есть в датасете явных дубликатов
display(stat[stat.duplicated()]) # чуть больше деталей по дубликатам
stat = stat.drop_duplicates() # 71 полный дубликат очень мало - можно смело удалять

#перейдем к неявным дубликатам
stat['education'].value_counts() # еще раз смотрю каким образом дублируются значения
stat['education'] = stat['education'].str.lower()
stat['education'].value_counts() # проверяем результат

54


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,2194.220567,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи
4182,1,2194.220567,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,142594,свадьба
4851,0,2194.220567,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба
5557,0,2194.220567,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
7808,0,2194.220567,57,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
8583,0,2194.220567,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,118514,дополнительное образование
9238,2,2194.220567,34,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для сдачи
9528,0,2194.220567,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,118514,операции со своей недвижимостью
9627,0,2194.220567,56,среднее,1,женат / замужем,0,F,пенсионер,0,118514,операции со своей недвижимостью
10462,0,2194.220567,62,среднее,1,женат / замужем,0,F,пенсионер,0,118514,покупка коммерческой недвижимости


среднее                15188
высшее                  5251
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64

Сначала убрал явные дубликаты из датасета, благо их было немного (~0.3%).

Для поиска неявных дубликатов я выбрал функцию ```value_counts```, чтобы сразу увидеть всю картину в целом. Т.к. дубликаты представляли собой лишь написание в разных регистрах, то устранить их было просто. Могли быть еще ошибки в написании, например.
Дубликаты в столбце возникли скорее всего из-за ввода руками, т.к. вряд ли есть смысл руками или автоматически как-либо исправлять эти значния.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [15]:
education_category = stat[['education', 'education_id']] # формируем новый датасет
education_category = education_category.drop_duplicates().reset_index(drop=True) # удаляем дубликаты, чтобы остались лишь id и категории
display(education_category)

# повторяем для семейного положения

family_status_category = stat[['family_status', 'family_status_id']]
family_status_category = family_status_category.drop_duplicates().reset_index(drop=True)
display(family_status_category)

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Я создал два словаря категорий. Теперь можно удалить лишние столбцы из основного датафрейма и, в случае чего, обращаться к созданным словарям.

In [16]:
stat = stat.drop(columns=['education', 'family_status'])
stat.info() # проверяю, что все получилось

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21470 entries, 0 to 21524
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21470 non-null  int64  
 1   days_employed     21470 non-null  float64
 2   dob_years         21470 non-null  int64  
 3   education_id      21470 non-null  int64  
 4   family_status_id  21470 non-null  int64  
 5   gender            21470 non-null  object 
 6   income_type       21470 non-null  object 
 7   debt              21470 non-null  int64  
 8   total_income      21470 non-null  int64  
 9   purpose           21470 non-null  object 
dtypes: float64(1), int64(6), object(3)
memory usage: 1.8+ MB


### Шаг 2.6. Категоризация дохода.

Добавлю в таблицу столбец с указанием категории дохода по следующим правилам:

доход - категория

0–30000 — **E**  
30001–50000 — **D**  
50001–200000 — **C**  
200001–1000000 — **B**  
1000001 и выше — **A**

In [17]:
def income_group(income):
    '''
    Возвращает категорию по сумме дохода по правилам:
    0–30000 — E
    30001–50000 — D
    50001–200000 — C
    200001–1000000 — B
    1000001 и выше — A
    '''
    if 0 <= income <= 30000:
        return 'E'
    if 30001 <= income <= 50000:
        return 'D'
    if 50001 <= income <= 200000:
        return 'C'
    if 200001 <= income <= 1000000:
        return 'B'
    return 'A'

# применим функцию для создания столбца с категорией
stat['total_income_category'] = stat['total_income'].apply(income_group)

stat.head(20) # проверю применение

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,2194.220567,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,926.185831,27,0,1,M,компаньон,0,255763,покупка жилья,B
6,0,2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем,B
7,0,152.779569,50,1,0,M,сотрудник,0,135823,образование,C
8,2,6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C


### Шаг 2.7. Категоризация целей кредита.

Теперь разберусь с целями получения кредита. Для этого создам функцию, которая определит по правилам категорию цели в заисимости от самой цели. Далее создам новый столбец в таблице, применив новую функцию.

In [18]:
stat.value_counts('purpose')

purpose
свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
операции с жильем                         652
покупка жилья для сдачи                   652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка своего жилья                      620
покупка недвижимости                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образовани

In [19]:
def purpose_group(purpose):
    # Принимает строку с целью и возвращает категорию цели
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'жиль' in purpose or 'недвижим' in purpose:
        return 'операции с недвижимостью'
    if 'образован' in purpose:
        return 'получение образования'

stat['purpose_category'] = stat['purpose'].apply(purpose_group) # добавляю новуй столбец с категорией цели в таблицу
print(stat.head(10)) # проверка
stat.info() # еще одна проверка

   children  days_employed  dob_years  education_id  family_status_id gender  \
0         1    8437.673028         42             0                 0      F   
1         1    4024.803754         36             1                 0      F   
2         0    5623.422610         33             1                 0      M   
3         3    4124.747207         32             1                 0      M   
4         0    2194.220567         53             1                 1      F   
5         0     926.185831         27             0                 1      M   
6         0    2879.202052         43             0                 0      F   
7         0     152.779569         50             1                 0      M   
8         2    6929.865299         35             0                 1      F   
9         0    2188.756445         41             1                 0      M   

  income_type  debt  total_income                     purpose  \
0   сотрудник     0        253875               покупк

### Ответы на вопросы.

Чтобы ответить на поставленные вопросы, я создам сводные таблицы где будут видны интересующие зависимости.

##### Вопрос 1:
Есть ли зависимость между количеством детей и возвратом кредита в срок? Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [20]:
stat_pivot_children = stat.pivot_table(index='children', values='debt', aggfunc='sum')
stat_pivot_children['%'] = stat_pivot_children['debt'] / stat['children'].value_counts() * 100 # добавил долю должников от всех людей в категории
display(stat_pivot_children)

,debt,%
children,,
0,1063,7.535800
1,445,9.163921
2,202,9.492481
3,27,8.181818
4,4,9.756098
5,0,0.000000


In [21]:
df_example = stat.pivot_table(index = 'children', values = 'debt', 
                            aggfunc = ['count', 'sum', 'mean', lambda x: 1 - x.mean()])
df_example.columns = ['Кол-во пользователей', 'Кол-во должников', '% должников', '% НЕдолжников']
df_example.style.format({'% должников': '{:.2%}', '% НЕдолжников': '{:.2%}'})

,Кол-во пользователей,Кол-во должников,% должников,% НЕдолжников
children,,,,
0,14106,1063,7.54%,92.46%
1,4856,445,9.16%,90.84%
2,2128,202,9.49%,90.51%
3,330,27,8.18%,91.82%
4,41,4,9.76%,90.24%
5,9,0,0.00%,100.00%


##### Вывод 1:




Как видно из таблицы в абсолютных значениях число заемщиков без детей, которые не вернули займ, разительно больше всех остальных вместе взятых. Это из-за того, что заемщиков без детей в принципе намного больше. Но если посмотреть на долю от общего числа заемщиков в указанных категориях, то оказывается, что заемщики с детьми испытывают проблемы с возвратом займа все же чаще.

Действительно, у заемщиков с детьми выше риск невозврата займа. Оно и понятно, ведь дети немалая статья расходов.

##### Вопрос 2:
Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [22]:
stat_pivot_family = stat.pivot_table(index='family_status_id', values='debt', aggfunc='sum')
stat_pivot_family['%'] = stat_pivot_family['debt'] / stat['family_status_id'].value_counts() * 100 # добавил долю должников от всех людей в категории
display(stat_pivot_family)
print('#'*40)
print(family_status_category) # легенда для упрощения чтения таблицы

,debt,%
family_status_id,,
0,931,7.542126
1,388,9.322441
2,63,6.569343
3,85,7.112971
4,274,9.750890


########################################
           family_status  family_status_id
0        женат / замужем                 0
1       гражданский брак                 1
2         вдовец / вдова                 2
3              в разводе                 3
4  Не женат / не замужем                 4


##### Вывод 2:

В данном случае видно, что заемщики, состоящие в т.н. гражданском браке и неженатые/незамужние чаще не возвращают взятые займы. На мой взгляд, это из-за менее спланированного и прогнозируемого образа жизни.
Можно было бы ответить, что люди не в отношениях чаще не возвращают займы, но в эту парадигму не ложатся категории вдовцов и разведенных. Вероятно, в этих случаях дело в незапланированном уменьшении доходов, но для релевантных выводов данного исследования недостаточно.

##### Вопрос 3:
Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [23]:
stat_pivot_income = stat.pivot_table(index=['total_income_category'], values='debt', aggfunc='sum')
stat_pivot_income['%'] = stat_pivot_income['debt'] / stat['total_income_category'].value_counts() * 100 # добавил долю должников от всех людей в категории
display(stat_pivot_income)

,debt,%
total_income_category,,
A,2,8.000000
B,356,7.062091
C,1360,8.483034
D,21,6.000000
E,2,9.090909


##### Вывод 3:

Представим лидеров по относительным значениям:
1. Уровень дохода до 30 000 р.
2. Уровень дохода 50 000 - 200 000 р.
3. Уровень дохода > 1 000 000 р.  

Слишком сильная разница, чтобы сказать, что есть очевидная зависимость невозвратов от уровня дохода.

##### Вопрос 4:
Как разные цели кредита влияют на его возврат в срок?

In [24]:
stat_pivot_purpose = stat.pivot_table(index='purpose_category', values='debt', aggfunc='sum')
stat_pivot_purpose['%'] = stat_pivot_purpose['debt'] / stat['purpose_category'].value_counts() * 100 # добавил долю должников от всех людей в категории
display(stat_pivot_purpose)

,debt,%
purpose_category,,
операции с автомобилем,403,9.354689
операции с недвижимостью,782,7.232036
получение образования,370,9.217738
проведение свадьбы,186,7.965739


##### Вывод 4:

В случае с целями займа, вывод я бы сформулировал следующим образом: долгосрочные займы возвращают чаще. Т.е. чем менее импульсивная, менее эмоциональная и менее продуманная/проработанная трата займа, тем выше шанс невозврата денег. Так, люди, берущие ипотеки, с большей вероятностью планируют все на годы-десятилетия вперед: как возвращать, как, возможно, зарабатывать на недвижимости.

## Общий вывод:

При оценке заемщика важно оценить наличие детей, семейное положение и цель займа. Самым надежным заемщиком будет являться замужний/женатый человек без детей, желающий приобрести жилье. Самым ненадежным - заемщик, не состоящий в браке, с одним-двумя детьми, желающий приобрести автомобиль.